In [1]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from scipy.misc import imresize
from scipy import ndimage

%matplotlib inline

import random
import csv
import re
import pickle
import json
from tqdm import tqdm
from natsort import natsorted

DATADIR = os.path.expanduser('~/data/kaggle/ultrasound-nerve-segmentation')
PROCDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/processed')
MODELDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/models')

In [2]:
with open(os.path.join(PROCDIR, 'train_files_map.pkl'), 'rb') as f:
    train_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'test_files_map.pkl'), 'rb') as f:
    test_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'conflicting_17.pkl'), 'rb') as f:
    conflicting = pickle.load(f)
    
keys_sorted = natsorted(list(train_files_map.keys()))

## presence dataset

In [29]:
# for images with no mask present, we apply the average mask of conflicting images
conflicting_masks = {}
for i in tqdm(conflicting.keys()):
    masks = []
    for j in conflicting[i]:
        file_info = train_files_map[keys_sorted[j]]
        mask = io.imread(os.path.join(file_info['folder'], file_info['mask_file']))
        masks.append(mask)
    masks = np.array(masks, dtype='float')
    conflicting_mask = np.mean(masks, axis=0) > 0.5
    conflicting_masks[i] = conflicting_mask

 16%|█▌        | 180/1154 [00:00<00:04, 238.31it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 1154/1154 [00:04<00:00, 234.73it/s]


In [30]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 96, 128
img_channels = 1

# 90/10 train/val split, by subject
data_images_train = []
data_presence_train = []
data_images_val = []
data_presence_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.1), replace=False)

for i in tqdm(range(len(keys_sorted))):
    key = keys_sorted[i]
    file_info = train_files_map[key]
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    img_filepath = os.path.join(folder, img_file)
    mask_filepath = os.path.join(folder, mask_file)
    image = io.imread(img_filepath)
    if i in conflicting:
        mask = conflicting_masks[i]
    else:
        mask = io.imread(mask_filepath)
    if np.sum(mask) > 0:
        presence = True
    else:
        presence = False
    image_resized = imresize(image, size=(img_rows, img_cols), interp='bilinear') / 255.0
    if subject in subjects_val:
        data_images_val.append(image_resized)
        data_presence_val.append(presence)
    else:
        data_images_train.append(image_resized)
        data_presence_train.append(presence)

data_images_train = np.expand_dims(np.array(data_images_train, dtype=np.float32), axis=3)
data_presence_train = np.array(data_presence_train, dtype=np.bool)
data_images_val = np.expand_dims(np.array(data_images_val, dtype=np.float32), axis=3)
data_presence_val = np.array(data_presence_val, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('presence shapes:', data_presence_train.shape, data_presence_val.shape)

  1%|          | 42/5635 [00:00<01:28, 63.16it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [01:32<00:00, 61.20it/s]


image shapes: (5157, 96, 128, 1) (478, 96, 128, 1)
presence shapes: (5157,) (478,)


In [31]:
multiplier = 5

data_images_train_augment = []
data_presence_train_augment = []
for i in tqdm(range(data_images_train.shape[0])):
    data_images_train_augment.append(data_images_train[i])
    data_presence_train_augment.append(data_presence_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_images_train.shape[1] * -0.125), 
                                 int(data_images_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_images_train.shape[2] * -0.125), 
                                 int(data_images_train.shape[2] * 0.125))
        image_aug = ndimage.interpolation.shift(data_images_train[i], (y_shift, x_shift, 0), order=0)
        presence = data_presence_train[i]
        data_images_train_augment.append(image_aug)
        data_presence_train_augment.append(presence)

data_images_train = np.array(data_images_train_augment, dtype=np.float32)
data_presence_train = np.array(data_presence_train_augment, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('presence shapes:', data_presence_train.shape, data_presence_val.shape)

100%|██████████| 5157/5157 [00:04<00:00, 1105.80it/s]


image shapes: (30942, 96, 128, 1) (478, 96, 128, 1)
presence shapes: (30942,) (478,)


In [32]:
with open(os.path.join(PROCDIR, 'data_train_val_presence_17c.pkl'), 'wb') as f:
    pickle.dump((data_images_train, data_presence_train, data_images_val, data_presence_val), f, protocol=4)

## segmentation dataset

In [33]:
# for images with no mask present, we apply the average mask of conflicting images
conflicting_masks = {}
for i in tqdm(conflicting.keys()):
    masks = []
    for j in conflicting[i]:
        file_info = train_files_map[keys_sorted[j]]
        mask = io.imread(os.path.join(file_info['folder'], file_info['mask_file']))
        masks.append(mask)
    masks = np.array(masks, dtype='float')
    conflicting_mask = np.mean(masks, axis=0) > 0.5
    conflicting_masks[i] = conflicting_mask

 15%|█▌        | 177/1154 [00:00<00:04, 231.99it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 1154/1154 [00:05<00:00, 224.38it/s]


In [34]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 96, 128
img_channels = 1

# 90/10 train/val split, by subject
data_images_train = []
data_masks_train = []
data_images_val = []
data_masks_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.1), replace=False)

for i in tqdm(range(len(keys_sorted))):
    key = keys_sorted[i]
    file_info = train_files_map[key]
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    img_filepath = os.path.join(folder, img_file)
    mask_filepath = os.path.join(folder, mask_file)
    image = io.imread(img_filepath)
    if i in conflicting:
        mask = conflicting_masks[i]
    else:
        mask = io.imread(mask_filepath)
    image_resized = imresize(image, size=(img_rows, img_cols), interp='bilinear') / 255.0
    mask_resized = imresize(mask, size=(img_rows, img_cols), interp='bilinear').astype(np.bool)
    if subject in subjects_val:
        data_images_val.append(image_resized)
        data_masks_val.append(mask_resized)
    else:
        data_images_train.append(image_resized)
        data_masks_train.append(mask_resized)

data_images_train = np.expand_dims(np.array(data_images_train, dtype=np.float32), axis=3)
data_masks_train = np.array(data_masks_train, dtype=np.bool)
data_images_val = np.expand_dims(np.array(data_images_val, dtype=np.float32), axis=3)
data_masks_val = np.array(data_masks_val, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

  1%|          | 44/5635 [00:00<01:53, 49.26it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [01:42<00:00, 56.98it/s]


image shapes: (5155, 96, 128, 1) (480, 96, 128, 1)
mask shapes: (5155, 96, 128) (480, 96, 128)


In [35]:
multiplier = 5

data_images_train_augment = []
data_masks_train_augment = []
for i in tqdm(range(data_images_train.shape[0])):
    data_images_train_augment.append(data_images_train[i])
    data_masks_train_augment.append(data_masks_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_images_train.shape[1] * -0.125), 
                                 int(data_images_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_images_train.shape[2] * -0.125), 
                                 int(data_images_train.shape[2] * 0.125))
        image_aug = ndimage.interpolation.shift(data_images_train[i], (y_shift, x_shift, 0), order=0)
        mask_aug = ndimage.interpolation.shift(data_masks_train[i], (y_shift, x_shift), order=0)
        data_images_train_augment.append(image_aug)
        data_masks_train_augment.append(mask_aug)

data_images_train = np.array(data_images_train_augment, dtype=np.float32)
data_masks_train = np.array(data_masks_train_augment, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

100%|██████████| 5155/5155 [00:08<00:00, 616.93it/s]


image shapes: (30930, 96, 128, 1) (480, 96, 128, 1)
mask shapes: (30930, 96, 128) (480, 96, 128)


In [36]:
with open(os.path.join(PROCDIR, 'data_train_val_segment_17c.pkl'), 'wb') as f:
    pickle.dump((data_images_train, data_masks_train, data_images_val, data_masks_val), f, protocol=4)

## mask dataset

In [37]:
# 90/10 train/val split, by subject
data_masks_train = []
data_masks_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.1), replace=False)

for file_info in tqdm(train_files_map.values()):
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    mask_filepath = os.path.join(folder, mask_file)
    mask = io.imread(mask_filepath)
    
    if np.sum(mask) == 0:
        continue
        
    mask_resized = (imresize(mask, size=(img_rows, img_cols), interp='bilinear') / 255.0).astype(np.bool)
    if subject in subjects_val:
        data_masks_val.append(mask_resized)
    else:
        data_masks_train.append(mask_resized)

data_masks_train = np.array(data_masks_train, dtype=np.bool)
data_masks_val = np.array(data_masks_val, dtype=np.bool)

print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

  4%|▍         | 246/5635 [00:00<00:10, 506.00it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [00:07<00:00, 784.81it/s]

mask shapes: (2083, 96, 128) (240, 96, 128)


In [38]:
with open(os.path.join(PROCDIR, 'data_train_val_vae_17c.pkl'), 'wb') as f:
    pickle.dump((data_masks_train, data_masks_val), f, protocol=4)